In [24]:
import datasets
import transformers
from datasets import Dataset
import json

In [2]:
from transformers import RobertaTokenizerFast

# Use model 
tokenizer = RobertaTokenizerFast.from_pretrained("vinai/phobert-base")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token


def tokenize_function(examples):
    return tokenizer(examples["context"], padding="max_length", truncation=True)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PhobertTokenizer'. 
The class this function is called from is 'RobertaTokenizerFast'.


In [3]:
# Function read file train
def read_data(filePath):
    f = open(filePath,encoding='utf-8')
    fileRead = json.load(f)
    context = []
    question = []
    answer = []
    for object in fileRead:
        for qa in object["qas"]:
            if(qa["question"] is None):
               print(object["context"].lower())
            context.append(object["context"].lower())
            question.append(qa["question"].lower())
            answer.append(qa["answer"]["text"][0].lower())
    dataset = {
    "context": context,
    "question": question,
    "answer": answer
    }
    return dataset

In [4]:
dataset = read_data("./QA_data/qa_train.json")
eval_dataset = read_data("./QA_data/qa_eval.json")

In [5]:
dataset_train = Dataset.from_dict(dataset)
dataset_eval = Dataset.from_dict(eval_dataset)

In [6]:
dataset_train

Dataset({
    features: ['context', 'question', 'answer'],
    num_rows: 1001
})

In [7]:
batch_size=8  # change to 16 for full training
encoder_max_length=64
decoder_max_length=32

def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
  inputs = tokenizer(batch["question"], padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs = tokenizer(batch["answer"], padding="max_length", truncation=True, max_length=decoder_max_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["decoder_input_ids"] = outputs.input_ids
  batch["decoder_attention_mask"] = outputs.attention_mask
  batch["labels"] = outputs.input_ids.copy()

  # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
  # We have to make sure that the PAD token is ignored
  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch

train_data = dataset_train.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns= dataset_train.column_names
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)


val_data = dataset_eval.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=dataset_eval.column_names
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

In [8]:
from transformers import EncoderDecoderModel

phoBert2PhoBert = EncoderDecoderModel.from_encoder_decoder_pretrained("vinai/phobert-base", "vinai/phobert-base")

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForCausalLM were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['roberta.encoder.layer.7.crossattention.self.query.weight', 'roberta.encoder.layer.10.crossattention.self.query.weight', 'roberta.encod

In [9]:
# set special tokens
phoBert2PhoBert.config.decoder_start_token_id = tokenizer.bos_token_id
phoBert2PhoBert.config.eos_token_id = tokenizer.eos_token_id
phoBert2PhoBert.config.pad_token_id = tokenizer.pad_token_id

# sensible parameters for beam search
phoBert2PhoBert.config.vocab_size = phoBert2PhoBert.config.decoder.vocab_size
phoBert2PhoBert.config.max_length = 64
phoBert2PhoBert.config.min_length = 56
phoBert2PhoBert.config.no_repeat_ngram_size = 3
phoBert2PhoBert.config.early_stopping = True
phoBert2PhoBert.config.length_penalty = 2.0
phoBert2PhoBert.config.num_beams = 4

In [10]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, EncoderDecoderModel

In [11]:
# load rouge for validation
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

C:\Users\ADMIN\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  


In [12]:
# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    output_dir="./baseline",
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=80,
    predict_with_generate=True,
    overwrite_output_dir=True,
)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=phoBert2PhoBert,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)
trainer.train()

C:\Users\ADMIN\AppData\Roaming\Python\Python37\site-packages\transformers\optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
  0%|          | 0/10080 [00:00<?, ?it/s]You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
C:\Users\ADMIN\AppData\Roaming\Python\Python37\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anter

{'loss': 0.4626, 'learning_rate': 4.751984126984127e-05, 'epoch': 3.97}


                                                        
  5%|▍         | 500/10080 [1:21:22<20:22:37,  7.66s/it]

{'eval_loss': 23.7374267578125, 'eval_rouge2_precision': 0.0084, 'eval_rouge2_recall': 0.0221, 'eval_rouge2_fmeasure': 0.0117, 'eval_runtime': 459.8674, 'eval_samples_per_second': 0.402, 'eval_steps_per_second': 0.052, 'epoch': 3.97}


C:\Users\ADMIN\AppData\Roaming\Python\Python37\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
 10%|▉         | 1000/10080 [2:29:24<20:49:03,  8.25s/it] 

...


 99%|█████████▉| 10000/10080 [25:35:31<10:53,  8.17s/it]

{'eval_loss': 27.784379959106445, 'eval_rouge2_precision': 0.0043, 'eval_rouge2_recall': 0.0164, 'eval_rouge2_fmeasure': 0.0066, 'eval_runtime': 472.2021, 'eval_samples_per_second': 0.392, 'eval_steps_per_second': 0.051, 'epoch': 79.37}


C:\Users\ADMIN\AppData\Roaming\Python\Python37\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
100%|██████████| 10080/10080 [25:46:33<00:00,  9.21s/it]   

{'train_runtime': 92792.9874, 'train_samples_per_second': 0.863, 'train_steps_per_second': 0.109, 'train_loss': 0.026708488502735567, 'epoch': 80.0}


TrainOutput(global_step=10080, training_loss=0.026708488502735567, metrics={'train_runtime': 92792.9874, 'train_samples_per_second': 0.863, 'train_steps_per_second': 0.109, 'train_loss': 0.026708488502735567, 'epoch': 80.0})

In [14]:
trainer.save_model()

In [25]:
# Function read file test
def read_data_test(filePath):
    f = open(filePath,encoding='utf-8')
    fileRead = json.load(f)
    question = []
    answer = []
    for object in fileRead:
        question.append(object["question"].lower())
        answer.append(object["answer"].lower())
    dataset = {
    "question": question,
    "answer": answer
    }
    return dataset

In [26]:
model = EncoderDecoderModel.from_pretrained("C:\\Users\\ADMIN\\Desktop\\dataset\\IR_QA\\baseline\\checkpoint-10000")
test_data = read_data_test("./QA_data/qa_test.json")
test_data = Dataset.from_dict(test_data)
batch_size = 16  # change to 64 for full evaluation

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    # cut off at BERT max length 512
    inputs = tokenizer(batch["question"], padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    outputs = model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["question"])

pred_str = results["pred"]
label_str = results["answer"]


Map:   0%|          | 0/87 [00:00<?, ? examples/s]C:\Users\ADMIN\AppData\Roaming\Python\Python37\site-packages\transformers\generation\utils.py:1357: UserWarning: Using `max_length`'s default (64) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [27]:
rouge = datasets.load_metric("rouge")
rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

print(rouge_output)

Score(precision=0.004604832669527387, recall=0.013055369533849028, fmeasure=0.006500846777531212)


In [28]:
from nltk.translate.bleu_score import sentence_bleu
#function to get score of results
def getScore(ref, candi):
    score = sentence_bleu(ref, candi)
    return float('{:.4f}'.format(score))

In [29]:
reference= []
for i in test_data["answer"]:
    reference.append(i.split())

In [30]:
scores = []
for i in pred_str:
    scores.append(getScore(reference,i.split()))

C:\Users\ADMIN\AppData\Roaming\Python\Python37\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\ADMIN\AppData\Roaming\Python\Python37\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\ADMIN\AppData\Roaming\Python\Python37\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using 